In [1]:
import feature_extractor as fe 
from os import listdir
import scipy.io as spio
import numpy as np
import dataset
    
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [2]:
patients = list(range(1,8))
patient_scores = []
for patient_number in patients:
    print("******PATIENT NUMBER " + str(patient_number) + "*******")
    #get data and channels
    #if(patient_number <= 3):
    #    cc = 5
    #elif(patient_number <= 5):
    #    cc = 4
    #else:
    #    cc = 3
    cc = 3
    channels = dataset.get_channels(patient_number, cc , 200)
    data, labels = dataset.get_data(patient_number, channels)
    
    #normalize
    data = np.nan_to_num(data)
    max1 = np.max(data, axis=0)
    data2 = data/max1
    
    #split into train and val
    #train_data, train_labels, val_data, val_labels = dataset.train_val_split(np.nan_to_num(data2.astype(np.float64)), labels, .75)

    '''
    #### Random forests
    m_vals = list(range(10, 130, 5))
    depth = list(range(1, 7))

    max_acc = [0, 0, 0, 0, 0, 0] #d, m, train roc, train acc, test roc, test acc
    for d in depth:
        #print("DEPTH " + str(d))
        for m in m_vals:
            #print("ESTIMATOR " + str(m))
            clf = RandomForestClassifier(n_estimators=m, max_depth=d, max_features = 5, random_state=0)
            clf.fit(train_data, train_labels)
            trn_r = metrics.roc_auc_score(train_labels, clf.predict_proba(train_data)[:,1])
            trn_a = clf.score(train_data, train_labels)
            tst_r = metrics.roc_auc_score(val_labels, clf.predict_proba(val_data)[:,1])
            tst_a = clf.score(val_data, val_labels) 
            #if(trn_r > max_acc[2] and trn_a > max_acc[3] and tst_r > max_acc[4] and 
            #   tst_a > max_acc[5] and trn_r < 0.999 and trn_a < 0.999):
            if(trn_a > 0.90 and tst_a > 0.95):
                max_acc = [d, m, trn_r, trn_a, tst_r, tst_a]
    print(max_acc)
    patient_scores.append(max_acc)
    
            print("train roc " + str(metrics.roc_auc_score(train_labels, clf.predict_proba(train_data)[:,1])))
            print("train acc " + str(clf.score(train_data, train_labels)) )
            print("test roc " + str(metrics.roc_auc_score(val_labels, clf.predict_proba(val_data)[:,1])))
            print("test acc " + str(clf.score(val_data, val_labels)))
    
    ''' #max_acc[1] max_acc[0]
    
    kf = KFold(n_splits = 10, random_state=None, shuffle=False)
    clf = RandomForestClassifier(n_estimators= 100 , max_depth= 3, max_features = 5, random_state=0)
    for train_index, test_index in kf.split(data2):
        X_train, X_test = data2[train_index], data2[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        clf.fit(X_train, y_train)
        patient_scores.append([clf.score(X_train, y_train), metrics.roc_auc_score(y_train, clf.predict_proba(X_train)),
                               clf.score(X_test, y_test), metrics.roc_auc_score(y_test, clf.predict_proba(X_test))])

    print("training")
    print(metrics.classification_report(train_labels,clf.predict(train_data)))
    print(metrics.roc_auc_score(train_labels, clf.predict_proba(train_data)[:,1]))
    print("\nvalidation")
    print(metrics.classification_report(val_labels,clf.predict(val_data)))
    print(metrics.roc_auc_score(val_labels,clf.predict_proba(val_data)[:,1]))
    print(metrics.accuracy_score(val_labels,clf.predict(val_data)))
    print("\n")

    fpr, tpr, thresholds = metrics.roc_curve(val_labels, clf.predict_proba(val_data)[:,1], pos_label=1)
    
    
    path = "data/patient_"+str(patient_number)+"/test/"
    files = listdir(path)
    #print(str(len(files))+" number of points")

    import os
    import csv
    csv_path = 'patient_roc' + str(patient_number)+'.csv'
    f = open(csv_path, "w")
    f.truncate()
    f.close()
    with open(csv_path, 'a') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for i, file in enumerate(files):
            if file == ".DS_Store":
                continue
            #print(str(i+1)+"/"+str(len(files))+": extracting "+file)
            file_data = spio.loadmat(path+file)["data"]
            vec = np.nan_to_num(fe.extract_feature2(file_data[:,channels]))
            vec = np.nan_to_num(vec/max1)
            vec = np.nan_to_num(vec.astype(np.float32))
            filename = file[:-4].replace("test_", "")
            score = clf.predict_proba(vec.reshape(-1, 1).T)[0,1]
            pred = [filename, score]
            #print(pred)

            spamwriter.writerow(pred)



******PATIENT NUMBER 1*******
218 number of points


C:\Users\Aasta\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 3, using nperseg = 3
  .format(nperseg, input_length))


directory data shape (218, 144)
600 number of points
directory data shape (600, 144)


C:\Users\Aasta\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
patient_number = 5
channels = dataset.get_channels(patient_number, 3, 200)
print(channels)
data, labels = dataset.get_data(patient_number, channels)


In [ ]:
print(data.shape)
print(labels.shape)
data = np.nan_to_num(data)
#std = np.std(data, axis=0)
#mean = np.mean(data, axis=0)
max1 = np.max(data, axis=0)
data2 = data/max1
data = np.nan_to_num((data-mean)/std)

#from sklearn.decomposition import PCA
#pca = PCA(n_components=50, whiten = True)
#pca.fit(data)
#data2 = pca.transform(data)

train_data, train_labels, val_data, val_labels = dataset.train_val_split(np.nan_to_num(data2.astype(np.float64)), labels, .75)

print(train_data.shape)
print(train_labels.shape)
print(val_data.shape)
print(val_labels.shape)

print(np.isnan(train_data).any())

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
#### SVM
#clf = SVC(C=10, gamma=1000, kernel='rbf',
 #  max_iter=10000000, probability=True)

    
#### Random forests
m_vals = list(range(10, 130, 10))
depth = list(range(1, 6))
train_roc = []
test_acc = []
test_roc = []
max_acc = [0, 0, 0, 0, 0, 0] #d, m, train roc, train acc, test roc, test acc
for d in depth:
    #print("DEPTH " + str(d))
    for m in m_vals:
        #print("ESTIMATOR " + str(m))
        clf = RandomForestClassifier(n_estimators=m, max_depth=d, max_features = 5, random_state=0)
        clf.fit(train_data, train_labels)
        trn_r = metrics.roc_auc_score(train_labels, clf.predict_proba(train_data)[:,1])
        trn_a = clf.score(train_data, train_labels)
        tst_r = metrics.roc_auc_score(val_labels, clf.predict_proba(val_data)[:,1])
        tst_a = clf.score(val_data, val_labels) 
        if(trn_r > max_acc[2] and trn_a > max_acc[3] and tst_r > max_acc[4] and 
           tst_a > max_acc[5] and trn_r < 0.999 and trn_a < 0.999):
            max_acc = [d, m, trn_r, trn_a, tst_r, tst_a]
print(max_acc)
'''
        print("train roc " + str(metrics.roc_auc_score(train_labels, clf.predict_proba(train_data)[:,1])))
        print("train acc " + str(clf.score(train_data, train_labels)) )
        print("test roc " + str(metrics.roc_auc_score(val_labels, clf.predict_proba(val_data)[:,1])))
        print("test acc " + str(clf.score(val_data, val_labels)))
'''
    
clf = RandomForestClassifier(n_estimators=max_acc[1], max_depth=max_acc[0], max_features = 5, random_state=0)
clf.fit(train_data, train_labels)

   
print("training")
print(metrics.classification_report(train_labels,clf.predict(train_data)))
print(metrics.roc_auc_score(train_labels, clf.predict_proba(train_data)[:,1]))
print("\nvalidation")
print(metrics.classification_report(val_labels,clf.predict(val_data)))
print(metrics.roc_auc_score(val_labels,clf.predict_proba(val_data)[:,1]))
print(metrics.accuracy_score(val_labels,clf.predict(val_data)))
print("\n")

fpr, tpr, thresholds = metrics.roc_curve(val_labels, clf.predict_proba(val_data)[:,1], pos_label=1)
import matplotlib.pyplot as plt
plt.plot(fpr,tpr)
print(thresholds)



In [ ]:
path = "data/patient_"+str(patient_number)+"/test/"
files = listdir(path)
print(str(len(files))+" number of points")

import os
import csv
csv_path = 'patient_roc' + str(patient_number)+'.csv'
f = open(csv_path, "w")
f.truncate()
f.close()
with open(csv_path, 'a') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i, file in enumerate(files):
        if file == ".DS_Store":
            continue
        print(str(i+1)+"/"+str(len(files))+": extracting "+file)
        file_data = spio.loadmat(path+file)["data"]
        vec = np.nan_to_num(fe.extract_feature2(file_data[:,channels]))
        vec = np.nan_to_num(vec/max1)
        vec = np.nan_to_num(vec.astype(np.float32))
        filename = file[:-4].replace("test_", "")
        score = clf.predict_proba(vec.reshape(-1, 1).T)[0,1]
        pred = [filename, score]
        print(pred)

        spamwriter.writerow(pred)


In [ ]:
###concatenate all files
import pandas as pd
files = ["patient_roc1.csv","patient_roc2.csv","patient_roc3.csv",
         "patient_roc4.csv","patient_roc5.csv","patient_roc6.csv","patient_roc7.csv"]

out = open("patient.csv", "w")
out.truncate()
out.close()

with open("patient.csv", 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id","prediction"])
    for file in files:
        with open(file, 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in reader:
                writer.writerow(row)

In [ ]:
#### bagging
#rf_trn = RandomForestClassifier(n_estimators = m, criterion = "gini")           
#rf_trn = rf_trn.fit(train_data, train_labels)
#y_pred = rf_trn.predict(val_data)

###Adaboost
#d = 5
#wc = 50
'''
classifier_vals = [5, 10, 25, 50, 100, 150, 200]
acc_trn = []
for wc in classifier_vals:
#tree_adaboost = DecisionTreeClassifier(max_depth=d) #, random_state = 0
    ab_trn = AdaBoostClassifier(base_estimator = clf, n_estimators = wc)           
    ab_trn = ab_trn.fit(train_data, train_labels)
    y_pred = ab_trn.predict(val_data)
    acc_trn.append(ab_trn.score(train_data, train_labels))

    
plt.figure()
plt.plot(classifier_vals, acc_trn, color = 'red')
plt.title('Adaboost: acc vs weak classifiers')
   # plt.xticks(np.arange(min(classifier_vals), max(classifier_vals)+1, 10))
plt.xlabel('weak classifiers')
plt.ylabel('accuracy')

from sklearn import metrics
print("training")
print(metrics.classification_report(ab_trn.predict(train_data), train_labels))
print(metrics.roc_auc_score(ab_trn.predict(train_data), train_labels))
print("\nvalidation")
print(metrics.classification_report(y_pred, val_labels))
print(metrics.roc_auc_score(y_pred, val_labels))
print("\n")
'''